In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report

print("Loading labeled data...")
data_path = '../../data/eurusd_h1_labeled.csv'
df = pd.read_csv(data_path, index_col='time', parse_dates=True)
df.dropna(inplace=True)
print("Data loaded successfully.")

In [ ]:
features = [col for col in df.columns if col not in ['open', 'high', 'low', 'close', 'volume', 'target']]
X = df[features]
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)
y_train_mapped = y_train.replace({-1: 2})
y_test_mapped = y_test.replace({-1: 2})
print("Data prepared for tuning.")

param_grid = {
    'n_estimators': [200, 500],             # More trees
    'learning_rate': [0.01, 0.05],          # Smaller learning rate
    'max_depth': [3, 5],                    # Shallower, more robust trees
    'gamma': [1, 5],                        # Stronger regularization
    'reg_lambda': [5, 10],                  # Stronger L2 regularization
    'subsample': [0.7, 0.9],                # Randomly sample data
    'colsample_bytree': [0.7, 0.9]          # Randomly sample features/indicators
}

In [ ]:
model = xgb.XGBClassifier(
    objective='multi:softprob',
    num_class=3,
    use_label_encoder=False,
    eval_metric='mlogloss'
)

grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='f1_weighted',
    n_jobs=-1,
    cv=3,
    verbose=2
)

print("Starting Hyperparameter Search... This may take a long time.")
grid_search.fit(X_train, y_train_mapped)
print("Search complete.")

In [ ]:
print("\n--- Best Parameters Found ---")
print(grid_search.best_params_)

In [ ]:
print("\n--- Performance of the Best Model on the Test Set ---")
best_model = grid_search.best_estimator_
y_pred_mapped = best_model.predict(X_test)
y_pred = pd.Series(y_pred_mapped).replace({2: -1}).values
print(classification_report(y_test, y_pred, zero_division=0))